In [15]:
import pandas as pd
import os
import concurrent.futures

pd.set_option('display.max_rows', 1000)

In [7]:
summary_file = os.path.join("data", "summary")
summary = pd.read_csv(summary_file)

In [37]:
summary.name

0               ABRCharacter
1                  ABRDragon
2                  Abilities
3               AbilityGroup
4        AbilityLimitedGroup
5                Adventurers
6              AffinityBonus
7                        Art
8       CampaignQuestHeaders
9                  CargoTest
10             CastleStories
11          ChainCoAbilities
12           CharaLimitBreak
13               CoAbilities
14                    Comics
15                Conditions
16               Consumables
17          DailyRewardChest
18                   Dragons
19               DropRewards
20              EndeavorSets
21           EndeavorSetsRow
22          EndeavorSetsTest
23               EnemiesTest
24            EnemyPageLinks
25         EventQuestHeaders
26                    Events
27                Facilities
28                     Gifts
29               LoadingTips
30               MCMaterials
31                MCNodeType
32                   MCNodes
33        MaintenanceNotices
34           M

In [32]:
weapon_tables = summary[(summary.name.str.contains("Weapon")) | (summary.name == "Materials")].sort_values("records_pulled",ascending=False)
weapon_tables

,name,records_pulled,start_time,end_time,duration,url,errors
53,Weaponsv1,1139,1.609973e+09,1.609973e+09,0.848851,https://dragalialost.gamepedia.com/Special:Car...,NaN
47,WeaponBodyBuildupGroup,1085,1.609973e+09,1.609973e+09,0.392264,https://dragalialost.gamepedia.com/Special:Car...,NaN
51,WeaponSkin,695,1.609973e+09,1.609973e+09,0.498649,https://dragalialost.gamepedia.com/Special:Car...,NaN
50,WeaponPassiveAbility,415,1.609973e+09,1.609973e+09,0.335351,https://dragalialost.gamepedia.com/Special:Car...,NaN
36,Materials,354,1.609973e+09,1.609973e+09,0.295614,https://dragalialost.gamepedia.com/Special:Car...,NaN
52,Weapons,262,1.609973e+09,1.609973e+09,0.605974,https://dragalialost.gamepedia.com/Special:Car...,NaN
48,WeaponBodyBuildupLevel,220,1.609973e+09,1.609973e+09,0.217645,https://dragalialost.gamepedia.com/Special:Car...,NaN
49,WeaponBodyRarity,6,1.609973e+09,1.609973e+09,0.229509,https://dragalialost.gamepedia.com/Special:Car...,NaN


In [33]:
wep_table_list = weapon_tables.name.to_list()

def get_file(table):
    filename = os.path.join("data", f"{table}.csv")
    return {table: pd.read_csv(filename)}

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(get_file, table) for table in wep_table_list]
    weapon_dfs = [f.result() for f in futures]
    weapon_dfs = {k: v for d in weapon_dfs for k,v in d.items()}

weapon_dfs.keys()

dict_keys(['Weaponsv1', 'WeaponBodyBuildupGroup', 'WeaponSkin', 'WeaponPassiveAbility', 'Materials', 'Weapons', 'WeaponBodyBuildupLevel', 'WeaponBodyRarity'])

In [36]:
for table, df in weapon_dfs.items():
    print(f"{table}:\n{[i for i in df.columns if 'Id' in i]}\n")

Weaponsv1:
['Id', 'BaseId', 'FormId', 'TypeId', 'ElementalTypeId', 'VariationId', 'CraftNodeId', 'ParentCraftNodeId', 'CraftGroupId', 'MainWeaponId', 'AcquiredWeaponId1', 'AcquiredWeaponId2', 'AvailabilityId']

WeaponBodyBuildupGroup:
['Id', 'WeaponBodyBuildupGroupId', 'BuildupPieceTypeId', 'BuildupMaterialId1', 'BuildupMaterialId2', 'BuildupMaterialId3', 'BuildupMaterialId4', 'BuildupMaterialId5', 'BuildupMaterialId6', 'BuildupMaterialId7', 'BuildupMaterialId8', 'BuildupMaterialId9', 'BuildupMaterialId10']

WeaponSkin:
['Id', 'WeaponTypeId', 'ElementalTypeId', 'BaseId', 'VariationId', 'FormId', 'DecBaseId', 'DecVariationId', 'BulletBaseId', 'BulletVariationId']

WeaponPassiveAbility:
['Id', 'WeaponPassiveAbilityGroupId', 'WeaponTypeId', 'ElementalTypeId', 'SortId', 'AbilityId', 'RewardWeaponSkinId1', 'RewardWeaponSkinId2', 'UnlockMaterialId1', 'UnlockMaterialId2', 'UnlockMaterialId3', 'UnlockMaterialId4', 'UnlockMaterialId5']

Materials:
['Id', 'QuestEventId', 'SortId']

Weapons:
['Id